In [3]:
# =============================================================================
#ユーザーが任意のファイルを指定し、各種PDF操作のボタンを押すと、
#処理が施されたPDFファイルが自身のDownloadsフォルダにアウトプットされる。
# =============================================================================

import os
import sys
import pypdf
import tkinter
import tkinter.filedialog
from tkinter import messagebox
from pypdf import PdfReader, PdfWriter
from pypdf import PdfMerger
import traceback
from functools import wraps
import functools
from pdf2docx import Converter

# =============================================================================
#グローバル変数の定義
# =============================================================================

# 現在のユーザー名を取得
user_name = os.getlogin()

# ダウンロードフォルダのパス
output_path = os.path.join(os.path.expanduser("~"), "Downloads")

# =============================================================================
#関数 file_select で取得した変数file_pathを引数として使用する
# =============================================================================

#GUIの基本サイズ
root = tkinter.Tk()
root.title(u"PDF operation")
root.geometry("500x450")

#ファイルを選択する関数
def file_select():
    global file_path
    idir= os.path.join(user_name, "Desktop")
    filetype = [("PDF","*.pdf"), ("all", "*")]
    file_path = tkinter.filedialog.askopenfilename(filetypes = filetype, initialdir = idir)
    input_box.insert(tkinter.END, file_path)
    return file_path

#フォルダを選択する関数
def directory_select():
    global folder_path
    idir= os.path.join(user_name, "Desktop")
    folder_path.set(tkinter.filedialog.askdirectory())
    input_box.insert(tkinter.END, folder_path.get())
    return folder_path

#選択したファイルを削除する関数
def DeleteEntryValue(event):
    #エントリーの中身を削除
    input_box.delete(0, tkinter.END)

#ファイルを選択するbox
input_box = tkinter.Entry(width=80)
input_box.place(x=10, y=10)

#ファイルを選択する参照ボタン
button_ref = tkinter.Button(text="ファイル選択", command=file_select)
button_ref.place(x=10, y=35)

#フォルダを選択する参照ボタン
button_ref2 = tkinter.Button(text="フォルダ選択", command=directory_select)
button_ref2.place(x=10, y=305)

#削除ボタン
button_del = tkinter.Button(text=u'削除', width= 8)
button_del.bind("<Button-1>", DeleteEntryValue) 
#左クリック（<Button-1>）されると，DeleteEntryValue関数を呼び出すようにバインド
button_del.place(x=85, y=35)

#結合使用時の注意ラベル
Static1 = tkinter.Label(text = u"PDFを結合する際はフォルダを選択してください", foreground='#ff0000')
Static1.place(x=85, y=308)

file_path = ""
folder_path = tkinter.StringVar()
# =============================================================================
# PDFを操作する処理
# =============================================================================

#PDFを時計回りに回転
def pdf_rotate90(event, input_file,output_path):
    if not file_path == "":
        if input_file.endswith('.pdf'):
            filename = os.path.basename(input_file)
            output_file = os.path.join(output_path, filename)
            #PDFリーダーにバイナリを指定して読み込み、PDFライターの設定
            with open(input_file, 'rb') as file:
                reader = PdfReader(file)
                writer = PdfWriter()

                #読み込んだPDFの総ページをlenで取得し、一枚づつ90°回転させ、ライターへ書き込む
                for page_number in range(len(reader.pages)):
                    page = reader.pages[page_number]
                    page.rotate(90)
                    writer.add_page(page)

                #ライターへ書き込まれたPDFデータをoutputdirへ排出。withによってcloseの必要なし。
                with open(output_file, 'wb') as output:
                    writer.write(output)
                    
        messagebox.showinfo("成功", "処理が完了しました。")
    else:
        # エラーが発生した場合、以下のメッセージボックスを表示します。
        messagebox.showinfo("失敗","ファイルを指定してください")
                
#PDFを反時計回りに回転
def pdf_rotate270(event, input_file,output_path):
    if not file_path == "":
        if input_file.endswith('.pdf'):
            filename = os.path.basename(input_file)
            output_file = os.path.join(output_path, filename)
            #PDFリーダーにバイナリを指定して読み込み、PDFライターの設定
            with open(input_file, 'rb') as file:
                reader = PdfReader(file)
                writer = PdfWriter()

                #読み込んだPDFの総ページをlenで取得し、一枚づつ90°回転させ、ライターへ書き込む
                for page_number in range(len(reader.pages)):
                    page = reader.pages[page_number]
                    page.rotate(270)
                    writer.add_page(page)

                #ライターへ書き込まれたPDFデータをoutputdirへ排出。withによってcloseの必要なし。
                with open(output_file, 'wb') as output:
                    writer.write(output)
                    
        messagebox.showinfo("成功", "処理が完了しました。")
    else:
        # エラーが発生した場合、以下のメッセージボックスを表示します。
        messagebox.showinfo("失敗","ファイルを指定してください")

#pdfファイルの分割
def pdf_split(event, input_file,output_path):
    if not file_path == "":
        if input_file.endswith('.pdf'):
            filename = os.path.basename(input_file)
            output_file = os.path.join(output_path, filename)
            #PDFリーダーにバイナリを指定して読み込み、PDFライターの設定
            with open(input_file, 'rb') as file:
                reader = pypdf.PdfReader(input_file)
                #分割したいファイルのページを一枚づつ書き出し
                for i, page in enumerate(reader.pages):
                    writer = pypdf.PdfWriter()
                    writer.add_page(page)
                    with open(f"{output_file}_{i}.pdf", "wb") as fp:
                        writer.write(fp)
        messagebox.showinfo("成功", "処理が完了しました。")
    else:
        # エラーが発生した場合、以下のメッセージボックスを表示します。
        messagebox.showinfo("失敗","ファイルを指定してください")

#PDFファイルをwordファイルへ変換
def pdf_change_to_docx(event, input_file,output_path):
    if not file_path == "":
        if input_file.endswith('.pdf'):
            filename = os.path.basename(input_file)
            output_file = os.path.join(output_path, filename)
            #PDFリーダーにバイナリを指定して読み込み、PDFライターの設定
            cv = Converter(input_file)
            docx_file = output_file.replace(".pdf", ".docx")
            cv.convert(docx_file)
            cv.close
        messagebox.showinfo("成功", "処理が完了しました。")
    else:
        # エラーが発生した場合、以下のメッセージボックスを表示します。
        messagebox.showinfo("失敗","ファイルを指定してください")

#複数のPDFを結合する
def pdf_merge(event, input_dir, output_dir):
    if not folder_path.get() == "":
        #マージするためのPdfMergerオブジェクトを作成
        merger = PdfMerger()
        #inputフォルダ内のファイルを取得
        for filename in os.listdir(input_dir):
            #拡張子が.pdfのファイルをinput,outputディレクトリと同期
            if filename.endswith('.pdf'):
                input_path = os.path.join(input_dir, filename)
                #ファイルを連結
                merger.append(input_path)
        #出力ファイルに保存
        with open(os.path.join(output_dir, "結合後.pdf"), "wb") as output_pdf:
            merger.write(output_pdf)
        messagebox.showinfo("成功", "処理が完了しました。")
    else:
        # エラーが発生した場合、以下のメッセージボックスを表示します。
        messagebox.showinfo("失敗","フォルダを指定してください")

# =============================================================================
#PDFを操作するメインボタン
# =============================================================================

#rotate_90 ボタン
button_90 = tkinter.Button(text=u'回転(時計回り)', command=functools.partial(pdf_rotate90, None, file_path, output_path), width= 25, height=5)
button_90.bind("<Button-1>", lambda event: pdf_rotate90(event, file_path, output_path)) 
button_90.place(x=40, y=80)

#rotate_270 ボタン
button_270 = tkinter.Button(text=u'回転(反時計回り)', command=functools.partial(pdf_rotate270, None, file_path, output_path), width= 25, height=5)
button_270.bind("<Button-1>", lambda event: pdf_rotate270(event, file_path, output_path)) 
button_270.place(x=270, y=80)

#split ボタン
button_me = tkinter.Button(text=u'分割', command=functools.partial(pdf_split, None, file_path, output_path),width= 25, height=5)
button_me.bind("<Button-1>", lambda event: pdf_split(event, file_path, output_path)) 
button_me.place(x=40, y=180)

#change_word ボタン
button_sp = tkinter.Button(text=u'word変換', command=functools.partial(pdf_change_to_docx, None, file_path, output_path), width= 25, height=5)
button_sp.bind("<Button-1>", lambda event: pdf_change_to_docx(event, file_path, output_path)) 
button_sp.place(x=270, y=180)

#marge ボタン
button_merge = tkinter.Button(text=u'PDF結合)', command=lambda: pdf_merge(None, folder_path.get(), output_path), width= 25, height=5)
button_merge.bind("<Button-1>",  lambda event: pdf_merge(event, folder_path.get(), output_path)) 
button_merge.place(x=40, y=350)
        
root.mainloop()

[INFO] Start to convert C:/Users/homas/Desktop/test用PDF.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...
[INFO] [3/4] Parsing pages...
[INFO] (1/5) Page 1
[INFO] (2/5) Page 2
[INFO] (3/5) Page 3
[INFO] (4/5) Page 4
[INFO] (5/5) Page 5
[INFO] [4/4] Creating pages...
[INFO] (1/5) Page 1
[INFO] (2/5) Page 2
[INFO] (3/5) Page 3
[INFO] (4/5) Page 4
[INFO] (5/5) Page 5
[INFO] Terminated in 2.69s.
